In [1]:
import pandas as pd
import numpy as np
from torchvision.datasets import Caltech256

import sys
import os

# Get the parent directory and add it to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from library.taxonomy_constructors import (
    SyntheticTaxonomy,
    CrossPredictionsTaxonomy,
)

In [2]:
# Load Caltech256 dataset information
caltech256_labels = Caltech256(root="../datasets/caltech256", download=False).categories

# Create synthetic taxonomy with same parameters as domain-shifted variant
caltech256_2d_variant_domain_shifted_synthetic_taxonomy = (
    SyntheticTaxonomy.create_synthetic_taxonomy(
        num_atomic_concepts=257,
        num_domains=2,
        domain_class_count_mean=200,
        domain_class_count_variance=10,
        concept_cluster_size_mean=2,
        concept_cluster_size_variance=1,
        no_prediction_class=True,
        atomic_concept_labels=caltech256_labels,
        relationship_type="true",
    )
)

In [3]:
# Load prediction data for domain-shifted variant
caltech256_2d_variant_domain_shifted_df_A = pd.read_csv(
    "../data/caltech256_2domain_variant_domain_shifted_A_predictions.csv"
)
caltech256_2d_variant_domain_shifted_df_B = pd.read_csv(
    "../data/caltech256_2domain_variant_domain_shifted_B_predictions.csv"
)

caltech256_2d_variant_domain_shifted_cross_domain_predictions = [
    (
        0,
        1,
        np.array(
            caltech256_2d_variant_domain_shifted_df_B["predictions_A_on_B"],
            dtype=np.intp,
        ),
    ),
    (
        1,
        0,
        np.array(
            caltech256_2d_variant_domain_shifted_df_A["predictions_B_on_A"],
            dtype=np.intp,
        ),
    ),
]

caltech256_2d_variant_domain_shifted_domain_targets = [
    (0, np.array(caltech256_2d_variant_domain_shifted_df_A["domain_A"], dtype=np.intp)),
    (1, np.array(caltech256_2d_variant_domain_shifted_df_B["domain_B"], dtype=np.intp)),
]

print(f"Domain A predictions shape: {caltech256_2d_variant_domain_shifted_df_A.shape}")
print(f"Domain B predictions shape: {caltech256_2d_variant_domain_shifted_df_B.shape}")

Domain A predictions shape: (2308, 2)
Domain B predictions shape: (2344, 2)


In [4]:
def evaluate_taxonomy(
    constructed_taxonomy, ground_truth_taxonomy, method_name, dataset_name, **params
):
    # Calculate metrics
    edr = constructed_taxonomy.edge_difference_ratio(ground_truth_taxonomy)
    precision, recall, f1 = constructed_taxonomy.precision_recall_f1(
        ground_truth_taxonomy
    )

    # Count relationships for analysis
    num_relationships = len(constructed_taxonomy.graph.edges())
    num_nodes = len(constructed_taxonomy.graph.nodes())

    results = {
        "method": method_name,
        "dataset": dataset_name,
        "edr": edr,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "num_relationships": num_relationships,
        "num_nodes": num_nodes,
        **params,  # Include method-specific parameters
    }

    return results

In [5]:
# MCFP Method Evaluation
mcfp_results_domain_shifted = []

caltech256_2d_variant_domain_shifted_mcfp_taxonomy = CrossPredictionsTaxonomy.from_cross_domain_predictions(
    cross_domain_predictions=caltech256_2d_variant_domain_shifted_cross_domain_predictions,
    domain_targets=caltech256_2d_variant_domain_shifted_domain_targets,
    domain_labels=caltech256_2d_variant_domain_shifted_synthetic_taxonomy.domain_labels,
    relationship_type="mcfp",
)

caltech256_2d_variant_domain_shifted_mcfp_result = evaluate_taxonomy(
    caltech256_2d_variant_domain_shifted_mcfp_taxonomy,
    caltech256_2d_variant_domain_shifted_synthetic_taxonomy,
    "mcfp",
    "caltech256_2domain_variant_domain_shifted",
)
mcfp_results_domain_shifted.append(caltech256_2d_variant_domain_shifted_mcfp_result)

print("MCFP Method Results:")
print(f"EDR: {caltech256_2d_variant_domain_shifted_mcfp_result['edr']:.4f}")
print(f"Precision: {caltech256_2d_variant_domain_shifted_mcfp_result['precision']:.4f}")
print(f"Recall: {caltech256_2d_variant_domain_shifted_mcfp_result['recall']:.4f}")
print(f"F1 Score: {caltech256_2d_variant_domain_shifted_mcfp_result['f1']:.4f}")

MCFP Method Results:
EDR: 0.7439
Precision: 0.6630
Recall: 0.3789
F1 Score: 0.4822


In [6]:
# Hypothesis Method Evaluation (with different upper bounds)
hypothesis_results_domain_shifted = {}

print("Evaluating Hypothesis Method...")
for upper_bound in range(1, 11):
    caltech256_2d_variant_domain_shifted_hypothesis_taxonomy = CrossPredictionsTaxonomy.from_cross_domain_predictions(
        cross_domain_predictions=caltech256_2d_variant_domain_shifted_cross_domain_predictions,
        domain_targets=caltech256_2d_variant_domain_shifted_domain_targets,
        domain_labels=caltech256_2d_variant_domain_shifted_synthetic_taxonomy.domain_labels,
        relationship_type="hypothesis",
        upper_bound=upper_bound,
    )

    result = evaluate_taxonomy(
        caltech256_2d_variant_domain_shifted_hypothesis_taxonomy,
        caltech256_2d_variant_domain_shifted_synthetic_taxonomy,
        "hypothesis",
        "caltech256_2domain_variant_domain_shifted",
        upper_bound=upper_bound,
    )

    hypothesis_results_domain_shifted[upper_bound] = result
    print(f"Upper bound {upper_bound}: EDR={result['edr']:.4f}, F1={result['f1']:.4f}")

# Find best EDR result
best_hypothesis_result = min(
    hypothesis_results_domain_shifted.values(), key=lambda x: x["edr"]
)
print(
    f"\nBest Hypothesis Result (Upper bound {best_hypothesis_result['upper_bound']}):"
)
print(f"EDR: {best_hypothesis_result['edr']:.4f}")
print(f"F1 Score: {best_hypothesis_result['f1']:.4f}")

Evaluating Hypothesis Method...
Upper bound 1: EDR=0.9510, F1=0.0632
Upper bound 2: EDR=0.7673, F1=0.4362
Upper bound 3: EDR=0.7004, F1=0.5468
Upper bound 4: EDR=0.6960, F1=0.5297
Upper bound 5: EDR=0.7033, F1=0.4914
Upper bound 6: EDR=0.7104, F1=0.4601
Upper bound 7: EDR=0.7170, F1=0.4269
Upper bound 8: EDR=0.7225, F1=0.4010
Upper bound 9: EDR=0.7265, F1=0.3826
Upper bound 10: EDR=0.7295, F1=0.3678

Best Hypothesis Result (Upper bound 4):
EDR: 0.6960
F1 Score: 0.5297


In [7]:
# Density Threshold Method Evaluation
density_threshold_results_domain_shifted = {}

print("Evaluating Density Threshold Method...")
for threshold in np.arange(0, 1.05, 0.05):
    threshold = round(threshold, 2).astype(float)

    caltech256_2d_variant_domain_shifted_density_taxonomy = CrossPredictionsTaxonomy.from_cross_domain_predictions(
        cross_domain_predictions=caltech256_2d_variant_domain_shifted_cross_domain_predictions,
        domain_targets=caltech256_2d_variant_domain_shifted_domain_targets,
        domain_labels=caltech256_2d_variant_domain_shifted_synthetic_taxonomy.domain_labels,
        relationship_type="density_threshold",
        threshold=threshold,
    )

    result = evaluate_taxonomy(
        caltech256_2d_variant_domain_shifted_density_taxonomy,
        caltech256_2d_variant_domain_shifted_synthetic_taxonomy,
        "density_threshold",
        "caltech256_2domain_variant_domain_shifted",
        threshold=threshold,
    )

    density_threshold_results_domain_shifted[threshold] = result

# Find best EDR result
best_density_result = min(
    density_threshold_results_domain_shifted.values(), key=lambda x: x["edr"]
)
print(f"Best Density Threshold Result (Threshold {best_density_result['threshold']}):")
print(f"EDR: {best_density_result['edr']:.4f}")
print(f"F1 Score: {best_density_result['f1']:.4f}")

Evaluating Density Threshold Method...
Best Density Threshold Result (Threshold 0.5):
EDR: 0.7029
F1 Score: 0.4989


In [8]:
# Simple Threshold Method Evaluation
simple_threshold_results_domain_shifted = {}

print("Evaluating Simple Threshold Method...")
for threshold in np.arange(0, 1.05, 0.05):
    threshold = round(threshold, 2).astype(float)

    caltech256_2d_variant_domain_shifted_simple_taxonomy = CrossPredictionsTaxonomy.from_cross_domain_predictions(
        cross_domain_predictions=caltech256_2d_variant_domain_shifted_cross_domain_predictions,
        domain_targets=caltech256_2d_variant_domain_shifted_domain_targets,
        domain_labels=caltech256_2d_variant_domain_shifted_synthetic_taxonomy.domain_labels,
        relationship_type="threshold",
        threshold=threshold,
    )

    result = evaluate_taxonomy(
        caltech256_2d_variant_domain_shifted_simple_taxonomy,
        caltech256_2d_variant_domain_shifted_synthetic_taxonomy,
        "threshold",
        "caltech256_2domain_variant_domain_shifted",
        threshold=threshold,
    )

    simple_threshold_results_domain_shifted[threshold] = result

# Find best EDR result
best_simple_result = min(
    simple_threshold_results_domain_shifted.values(), key=lambda x: x["edr"]
)
print(f"Best Simple Threshold Result (Threshold {best_simple_result['threshold']}):")
print(f"EDR: {best_simple_result['edr']:.4f}")
print(f"F1 Score: {best_simple_result['f1']:.4f}")

Evaluating Simple Threshold Method...
Best Simple Threshold Result (Threshold 0.15):
EDR: 0.7000
F1 Score: 0.5700


In [9]:
# Setup matplotlib for LaTeX output
import matplotlib

matplotlib.use("pgf")
import matplotlib.pyplot as plt

# LaTeX settings
plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "EB Garamond",
        "font.size": 11,
        "pgf.texsystem": "lualatex",
    }
)

In [10]:
# Plot 1: Hypothesis method precision-recall curve (domain-shifted variant)
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

# Extract data for hypothesis method
precisions = []
recalls = []
upper_bounds = []

for upper_bound, result in hypothesis_results_domain_shifted.items():
    precisions.append(result["precision"])
    recalls.append(result["recall"])
    upper_bounds.append(upper_bound)

# Create scatter plot with colormap
scatter = ax.scatter(recalls, precisions, c=upper_bounds, cmap="viridis", s=50)

ax.set_xlabel("Recall")
ax.set_ylabel("Precision")
ax.set_title("Caltech-256 2-Domain Variant Domain Shifted - Hypothesis Method")
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label("Upper Bound")

plt.tight_layout()
plt.savefig(
    "../../thesis/figures/hypothesis_method_precision_recall_domain_shifted.pgf",
    bbox_inches="tight",
)
plt.show()

/tmp/ipykernel_6501/2396567050.py:33: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [11]:
# Plot 2: Naive threshold method precision-recall curve (domain-shifted variant)
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

# Extract data for simple threshold method
precisions = []
recalls = []
thresholds = []

for threshold, result in simple_threshold_results_domain_shifted.items():
    precisions.append(result["precision"])
    recalls.append(result["recall"])
    thresholds.append(threshold)

# Create scatter plot with colormap
scatter = ax.scatter(recalls, precisions, c=thresholds, cmap="plasma", s=50)

ax.set_xlabel("Recall")
ax.set_ylabel("Precision")
ax.set_title("Caltech-256 2-Domain Variant Domain Shifted - Naive Threshold Method")
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label("Threshold")

plt.tight_layout()
plt.savefig(
    "../../thesis/figures/naive_threshold_method_precision_recall_domain_shifted.pgf",
    bbox_inches="tight",
)
plt.show()

/tmp/ipykernel_6501/3939321607.py:33: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [12]:
# Plot 3: Density threshold method precision-recall curve (domain-shifted variant)
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

# Extract data for density threshold method
precisions = []
recalls = []
thresholds = []

for threshold, result in density_threshold_results_domain_shifted.items():
    precisions.append(result["precision"])
    recalls.append(result["recall"])
    thresholds.append(threshold)

# Create scatter plot with colormap
scatter = ax.scatter(recalls, precisions, c=thresholds, cmap="inferno", s=50)

ax.set_xlabel("Recall")
ax.set_ylabel("Precision")
ax.set_title("Caltech-256 2-Domain Variant Domain Shifted - Density Threshold Method")
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label("Threshold")

plt.tight_layout()
plt.savefig(
    "../../thesis/figures/density_threshold_method_precision_recall_domain_shifted.pgf",
    bbox_inches="tight",
)
plt.show()

/tmp/ipykernel_6501/1295150003.py:33: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [13]:
# Create comprehensive results table for domain-shifted variant
import pandas as pd

# Prepare data for the table
table_data_domain_shifted = []

# Add MCFP result
mcfp_result = mcfp_results_domain_shifted[0]
table_data_domain_shifted.append(
    {
        "Method": "MCFP",
        "EDR": f"{mcfp_result['edr']:.3f}",
        "F1-score": f"{mcfp_result['f1']:.3f}",
        "Precision": f"{mcfp_result['precision']:.3f}",
        "Recall": f"{mcfp_result['recall']:.3f}",
        "Parameter": "N/A",
    }
)

# Add best results from parametric methods
methods = [
    ("Naive Thresholding", simple_threshold_results_domain_shifted, "threshold"),
    ("Density Thresholding", density_threshold_results_domain_shifted, "threshold"),
    ("Relationship Hypothesis", hypothesis_results_domain_shifted, "upper_bound"),
]

for method_name, results_dict, param_key in methods:
    # Find best EDR result
    best_result = min(results_dict.values(), key=lambda x: x["edr"])
    param_value = best_result[param_key]
    param_display = (
        f"{param_value:.2f}" if isinstance(param_value, float) else str(param_value)
    )

    table_data_domain_shifted.append(
        {
            "Method": method_name,
            "EDR": f"{best_result['edr']:.3f}",
            "F1-score": f"{best_result['f1']:.3f}",
            "Precision": f"{best_result['precision']:.3f}",
            "Recall": f"{best_result['recall']:.3f}",
            "Parameter": param_display,
        }
    )

# Create DataFrame
df_domain_shifted = pd.DataFrame(table_data_domain_shifted)

In [14]:
# Function to apply bold formatting to best values in each column
def apply_bold_to_best_values_domain_shifted(df):
    """Apply bold formatting to the best value in each metric column independently"""
    formatted_df = df.copy()

    # Find best EDR (lowest value)
    edr_values = [float(row["EDR"]) for row in table_data_domain_shifted]
    best_edr_value = min(edr_values)

    # Find best F1-score (highest value)
    f1_values = [float(row["F1-score"]) for row in table_data_domain_shifted]
    best_f1_value = max(f1_values)

    # Find best Precision (highest value)
    precision_values = [float(row["Precision"]) for row in table_data_domain_shifted]
    best_precision_value = max(precision_values)

    # Find best Recall (highest value)
    recall_values = [float(row["Recall"]) for row in table_data_domain_shifted]
    best_recall_value = max(recall_values)

    # Apply bold formatting to best values
    for idx, row in formatted_df.iterrows():
        if float(row["EDR"]) == best_edr_value:
            formatted_df.loc[idx, "EDR"] = f"\\textbf{{{row['EDR']}}}"

        if float(row["F1-score"]) == best_f1_value:
            formatted_df.loc[idx, "F1-score"] = f"\\textbf{{{row['F1-score']}}}"

        if float(row["Precision"]) == best_precision_value:
            formatted_df.loc[idx, "Precision"] = f"\\textbf{{{row['Precision']}}}"

        if float(row["Recall"]) == best_recall_value:
            formatted_df.loc[idx, "Recall"] = f"\\textbf{{{row['Recall']}}}"

    return formatted_df


# Apply bold formatting
df_formatted_domain_shifted = apply_bold_to_best_values_domain_shifted(
    df_domain_shifted
)

# Create styler and generate LaTeX table
styler_domain_shifted = df_formatted_domain_shifted.style.hide(axis="index")

latex_table_domain_shifted = styler_domain_shifted.to_latex(
    caption="Best EDR results for relationship discovery methods on Caltech-256 2-Domain Variant Domain Shifted dataset. For each method, the parameter values that yielded the lowest Edge Difference Ratio (EDR) are shown along with the corresponding performance metrics.",
    label="tab:relationship_methods_domain_shifted_results",
    column_format="lccccc",  # Left align method, center align metrics
    position="ht",
    position_float="centering",
    hrules=True,
)

# Save to file
with open(
    "../../thesis/figures/relationship_methods_domain_shifted_results.tex", "w"
) as f:
    f.write(latex_table_domain_shifted)

print(
    "Results table saved to ../../thesis/figures/relationship_methods_domain_shifted_results.tex"
)
print("\nSummary of Best Results:")
print(df_domain_shifted.to_string(index=False))

Results table saved to ../../thesis/figures/relationship_methods_domain_shifted_results.tex

Summary of Best Results:
                 Method   EDR F1-score Precision Recall Parameter
                   MCFP 0.744    0.482     0.663  0.379       N/A
     Naive Thresholding 0.700    0.570     0.615  0.531      0.15
   Density Thresholding 0.703    0.499     0.395  0.677      0.50
Relationship Hypothesis 0.696    0.530     0.440  0.665         4


In [15]:
# Compare with original variant (load original results for comparison)
print("=== Comparison with Original Variant ===")
print("\nDomain-Shifted Variant Results:")
for result in table_data_domain_shifted:
    print(f"{result['Method']}: EDR={result['EDR']}, F1={result['F1-score']}")

print("\n=== Analysis ===")
print(f"MCFP Method:")
print(f"  EDR: {mcfp_result['edr']:.4f}")
print(f"  F1:  {mcfp_result['f1']:.4f}")
print(
    f"  This suggests {'better' if mcfp_result['edr'] < 0.5 else 'worse'} performance than expected"
)

print(
    f"\nBest Overall Method (by EDR): {min(table_data_domain_shifted, key=lambda x: float(x['EDR']))['Method']}"
)
print(
    f"Best Overall Method (by F1):  {max(table_data_domain_shifted, key=lambda x: float(x['F1-score']))['Method']}"
)

=== Comparison with Original Variant ===

Domain-Shifted Variant Results:
MCFP: EDR=0.744, F1=0.482
Naive Thresholding: EDR=0.700, F1=0.570
Density Thresholding: EDR=0.703, F1=0.499
Relationship Hypothesis: EDR=0.696, F1=0.530

=== Analysis ===
MCFP Method:
  EDR: 0.7439
  F1:  0.4822
  This suggests worse performance than expected

Best Overall Method (by EDR): Relationship Hypothesis
Best Overall Method (by F1):  Naive Thresholding


In [16]:
# Load prediction data for domain-shifted variant
caltech256_2d_variant_domain_shifted_df_A = pd.read_csv(
    "../data/caltech256_2domain_variant_domain_shifted_A_predictions.csv"
)
caltech256_2d_variant_domain_shifted_df_B = pd.read_csv(
    "../data/caltech256_2domain_variant_domain_shifted_B_predictions.csv"
)

caltech256_2d_variant_domain_shifted_cross_domain_predictions = [
    (
        0,
        1,
        np.array(
            caltech256_2d_variant_domain_shifted_df_B["predictions_A_on_B"],
            dtype=np.intp,
        ),
    ),
    (
        1,
        0,
        np.array(
            caltech256_2d_variant_domain_shifted_df_A["predictions_B_on_A"],
            dtype=np.intp,
        ),
    ),
]

caltech256_2d_variant_domain_shifted_domain_targets = [
    (0, np.array(caltech256_2d_variant_domain_shifted_df_A["domain_A"], dtype=np.intp)),
    (1, np.array(caltech256_2d_variant_domain_shifted_df_B["domain_B"], dtype=np.intp)),
]

print(f"Domain A predictions shape: {caltech256_2d_variant_domain_shifted_df_A.shape}")
print(f"Domain B predictions shape: {caltech256_2d_variant_domain_shifted_df_B.shape}")

Domain A predictions shape: (2308, 2)
Domain B predictions shape: (2344, 2)


In [17]:
def evaluate_taxonomy(
    constructed_taxonomy, ground_truth_taxonomy, method_name, dataset_name, **params
):
    # Calculate metrics
    edr = constructed_taxonomy.edge_difference_ratio(ground_truth_taxonomy)
    precision, recall, f1 = constructed_taxonomy.precision_recall_f1(
        ground_truth_taxonomy
    )

    # Count relationships for analysis
    num_relationships = len(constructed_taxonomy.graph.edges())
    num_nodes = len(constructed_taxonomy.graph.nodes())

    results = {
        "method": method_name,
        "dataset": dataset_name,
        "edr": edr,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "num_relationships": num_relationships,
        "num_nodes": num_nodes,
        **params,  # Include method-specific parameters
    }

    return results

In [18]:
# MCFP Method Evaluation
mcfp_results_domain_shifted = []

caltech256_2d_variant_domain_shifted_mcfp_taxonomy = CrossPredictionsTaxonomy.from_cross_domain_predictions(
    cross_domain_predictions=caltech256_2d_variant_domain_shifted_cross_domain_predictions,
    domain_targets=caltech256_2d_variant_domain_shifted_domain_targets,
    domain_labels=caltech256_2d_variant_domain_shifted_synthetic_taxonomy.domain_labels,
    relationship_type="mcfp",
)

caltech256_2d_variant_domain_shifted_mcfp_result = evaluate_taxonomy(
    caltech256_2d_variant_domain_shifted_mcfp_taxonomy,
    caltech256_2d_variant_domain_shifted_synthetic_taxonomy,
    "mcfp",
    "caltech256_2domain_variant_domain_shifted",
)
mcfp_results_domain_shifted.append(caltech256_2d_variant_domain_shifted_mcfp_result)

print("MCFP Method Results:")
print(f"EDR: {caltech256_2d_variant_domain_shifted_mcfp_result['edr']:.4f}")
print(f"Precision: {caltech256_2d_variant_domain_shifted_mcfp_result['precision']:.4f}")
print(f"Recall: {caltech256_2d_variant_domain_shifted_mcfp_result['recall']:.4f}")
print(f"F1 Score: {caltech256_2d_variant_domain_shifted_mcfp_result['f1']:.4f}")

MCFP Method Results:
EDR: 0.7439
Precision: 0.6630
Recall: 0.3789
F1 Score: 0.4822


In [19]:
# Hypothesis Method Evaluation (with different upper bounds)
hypothesis_results_domain_shifted = {}

print("Evaluating Hypothesis Method...")
for upper_bound in range(1, 11):
    caltech256_2d_variant_domain_shifted_hypothesis_taxonomy = CrossPredictionsTaxonomy.from_cross_domain_predictions(
        cross_domain_predictions=caltech256_2d_variant_domain_shifted_cross_domain_predictions,
        domain_targets=caltech256_2d_variant_domain_shifted_domain_targets,
        domain_labels=caltech256_2d_variant_domain_shifted_synthetic_taxonomy.domain_labels,
        relationship_type="hypothesis",
        upper_bound=upper_bound,
    )

    result = evaluate_taxonomy(
        caltech256_2d_variant_domain_shifted_hypothesis_taxonomy,
        caltech256_2d_variant_domain_shifted_synthetic_taxonomy,
        "hypothesis",
        "caltech256_2domain_variant_domain_shifted",
        upper_bound=upper_bound,
    )

    hypothesis_results_domain_shifted[upper_bound] = result
    print(f"Upper bound {upper_bound}: EDR={result['edr']:.4f}, F1={result['f1']:.4f}")

# Find best EDR result
best_hypothesis_result = min(
    hypothesis_results_domain_shifted.values(), key=lambda x: x["edr"]
)
print(
    f"\nBest Hypothesis Result (Upper bound {best_hypothesis_result['upper_bound']}):"
)
print(f"EDR: {best_hypothesis_result['edr']:.4f}")
print(f"F1 Score: {best_hypothesis_result['f1']:.4f}")

Evaluating Hypothesis Method...
Upper bound 1: EDR=0.9510, F1=0.0632
Upper bound 2: EDR=0.7673, F1=0.4362
Upper bound 3: EDR=0.7004, F1=0.5468
Upper bound 4: EDR=0.6960, F1=0.5297
Upper bound 5: EDR=0.7033, F1=0.4914
Upper bound 6: EDR=0.7104, F1=0.4601
Upper bound 7: EDR=0.7170, F1=0.4269
Upper bound 8: EDR=0.7225, F1=0.4010
Upper bound 9: EDR=0.7265, F1=0.3826
Upper bound 10: EDR=0.7295, F1=0.3678

Best Hypothesis Result (Upper bound 4):
EDR: 0.6960
F1 Score: 0.5297


In [20]:
# Density Threshold Method Evaluation
density_threshold_results_domain_shifted = {}

print("Evaluating Density Threshold Method...")
for threshold in np.arange(0, 1.05, 0.05):
    threshold = round(threshold, 2).astype(float)

    caltech256_2d_variant_domain_shifted_density_taxonomy = CrossPredictionsTaxonomy.from_cross_domain_predictions(
        cross_domain_predictions=caltech256_2d_variant_domain_shifted_cross_domain_predictions,
        domain_targets=caltech256_2d_variant_domain_shifted_domain_targets,
        domain_labels=caltech256_2d_variant_domain_shifted_synthetic_taxonomy.domain_labels,
        relationship_type="density_threshold",
        threshold=threshold,
    )

    result = evaluate_taxonomy(
        caltech256_2d_variant_domain_shifted_density_taxonomy,
        caltech256_2d_variant_domain_shifted_synthetic_taxonomy,
        "density_threshold",
        "caltech256_2domain_variant_domain_shifted",
        threshold=threshold,
    )

    density_threshold_results_domain_shifted[threshold] = result

# Find best EDR result
best_density_result = min(
    density_threshold_results_domain_shifted.values(), key=lambda x: x["edr"]
)
print(f"Best Density Threshold Result (Threshold {best_density_result['threshold']}):")
print(f"EDR: {best_density_result['edr']:.4f}")
print(f"F1 Score: {best_density_result['f1']:.4f}")

Evaluating Density Threshold Method...
Best Density Threshold Result (Threshold 0.5):
EDR: 0.7029
F1 Score: 0.4989


In [21]:
# Simple Threshold Method Evaluation
simple_threshold_results_domain_shifted = {}

print("Evaluating Simple Threshold Method...")
for threshold in np.arange(0, 1.05, 0.05):
    threshold = round(threshold, 2).astype(float)

    caltech256_2d_variant_domain_shifted_simple_taxonomy = CrossPredictionsTaxonomy.from_cross_domain_predictions(
        cross_domain_predictions=caltech256_2d_variant_domain_shifted_cross_domain_predictions,
        domain_targets=caltech256_2d_variant_domain_shifted_domain_targets,
        domain_labels=caltech256_2d_variant_domain_shifted_synthetic_taxonomy.domain_labels,
        relationship_type="threshold",
        threshold=threshold,
    )

    result = evaluate_taxonomy(
        caltech256_2d_variant_domain_shifted_simple_taxonomy,
        caltech256_2d_variant_domain_shifted_synthetic_taxonomy,
        "threshold",
        "caltech256_2domain_variant_domain_shifted",
        threshold=threshold,
    )

    simple_threshold_results_domain_shifted[threshold] = result

# Find best EDR result
best_simple_result = min(
    simple_threshold_results_domain_shifted.values(), key=lambda x: x["edr"]
)
print(f"Best Simple Threshold Result (Threshold {best_simple_result['threshold']}):")
print(f"EDR: {best_simple_result['edr']:.4f}")
print(f"F1 Score: {best_simple_result['f1']:.4f}")

Evaluating Simple Threshold Method...
Best Simple Threshold Result (Threshold 0.15):
EDR: 0.7000
F1 Score: 0.5700


In [22]:
# Setup matplotlib for LaTeX output
import matplotlib

matplotlib.use("pgf")
import matplotlib.pyplot as plt

# LaTeX settings
plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "EB Garamond",
        "font.size": 11,
        "pgf.texsystem": "lualatex",
    }
)

In [23]:
# Plot 1: Hypothesis method precision-recall curve (domain-shifted variant)
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

# Extract data for hypothesis method
precisions = []
recalls = []
upper_bounds = []

for upper_bound, result in hypothesis_results_domain_shifted.items():
    precisions.append(result["precision"])
    recalls.append(result["recall"])
    upper_bounds.append(upper_bound)

# Create scatter plot with colormap
scatter = ax.scatter(recalls, precisions, c=upper_bounds, cmap="viridis", s=50)

ax.set_xlabel("Recall")
ax.set_ylabel("Precision")
ax.set_title("Caltech-256 2-Domain Variant Domain Shifted - Hypothesis Method")
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label("Upper Bound")

plt.tight_layout()
plt.savefig(
    "../../thesis/figures/hypothesis_method_precision_recall_domain_shifted.pgf",
    bbox_inches="tight",
)
plt.show()

/tmp/ipykernel_6501/2396567050.py:33: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [24]:
# Plot 2: Naive threshold method precision-recall curve (domain-shifted variant)
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

# Extract data for simple threshold method
precisions = []
recalls = []
thresholds = []

for threshold, result in simple_threshold_results_domain_shifted.items():
    precisions.append(result["precision"])
    recalls.append(result["recall"])
    thresholds.append(threshold)

# Create scatter plot with colormap
scatter = ax.scatter(recalls, precisions, c=thresholds, cmap="plasma", s=50)

ax.set_xlabel("Recall")
ax.set_ylabel("Precision")
ax.set_title("Caltech-256 2-Domain Variant Domain Shifted - Naive Threshold Method")
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label("Threshold")

plt.tight_layout()
plt.savefig(
    "../../thesis/figures/naive_threshold_method_precision_recall_domain_shifted.pgf",
    bbox_inches="tight",
)
plt.show()

/tmp/ipykernel_6501/3939321607.py:33: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [25]:
# Plot 3: Density threshold method precision-recall curve (domain-shifted variant)
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

# Extract data for density threshold method
precisions = []
recalls = []
thresholds = []

for threshold, result in density_threshold_results_domain_shifted.items():
    precisions.append(result["precision"])
    recalls.append(result["recall"])
    thresholds.append(threshold)

# Create scatter plot with colormap
scatter = ax.scatter(recalls, precisions, c=thresholds, cmap="inferno", s=50)

ax.set_xlabel("Recall")
ax.set_ylabel("Precision")
ax.set_title("Caltech-256 2-Domain Variant Domain Shifted - Density Threshold Method")
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label("Threshold")

plt.tight_layout()
plt.savefig(
    "../../thesis/figures/density_threshold_method_precision_recall_domain_shifted.pgf",
    bbox_inches="tight",
)
plt.show()

/tmp/ipykernel_6501/1295150003.py:33: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [29]:
# Create comprehensive results table for domain-shifted variant
import pandas as pd

# Prepare data for the table
table_data_domain_shifted = []

# Add MCFP result
mcfp_result = mcfp_results_domain_shifted[0]
table_data_domain_shifted.append(
    {
        "Method": "MCFP",
        "EDR": f"{mcfp_result['edr']:.3f}",
        "F1-score": f"{mcfp_result['f1']:.3f}",
        "Precision": f"{mcfp_result['precision']:.3f}",
        "Recall": f"{mcfp_result['recall']:.3f}",
        "Parameter": "N/A",
    }
)

# Add best results from parametric methods
methods = [
    ("Naive Thresholding", simple_threshold_results_domain_shifted, "threshold"),
    ("Density Thresholding", density_threshold_results_domain_shifted, "threshold"),
    ("Relationship Hypothesis", hypothesis_results_domain_shifted, "upper_bound"),
]

for method_name, results_dict, param_key in methods:
    # Find best EDR result
    best_result = min(results_dict.values(), key=lambda x: x["edr"])
    param_value = best_result[param_key]
    param_display = (
        f"{param_value:.2f}" if isinstance(param_value, float) else str(param_value)
    )

    table_data_domain_shifted.append(
        {
            "Method": method_name,
            "EDR": f"{best_result['edr']:.3f}",
            "F1-score": f"{best_result['f1']:.3f}",
            "Precision": f"{best_result['precision']:.3f}",
            "Recall": f"{best_result['recall']:.3f}",
            "Parameter": param_display,
        }
    )

# Create DataFrame
df_domain_shifted = pd.DataFrame(table_data_domain_shifted)


# Function to apply bold formatting to best values in each column
def apply_bold_to_best_values_domain_shifted(df):
    """Apply bold formatting to the best value in each metric column independently"""
    formatted_df = df.copy()

    # Find best EDR (lowest value)
    edr_values = [float(row["EDR"]) for row in table_data_domain_shifted]
    best_edr_value = min(edr_values)

    # Find best F1-score (highest value)
    f1_values = [float(row["F1-score"]) for row in table_data_domain_shifted]
    best_f1_value = max(f1_values)

    # Find best Precision (highest value)
    precision_values = [float(row["Precision"]) for row in table_data_domain_shifted]
    best_precision_value = max(precision_values)

    # Find best Recall (highest value)
    recall_values = [float(row["Recall"]) for row in table_data_domain_shifted]
    best_recall_value = max(recall_values)

    # Apply bold formatting to best values
    for idx, row in formatted_df.iterrows():
        if float(row["EDR"]) == best_edr_value:
            formatted_df.loc[idx, "EDR"] = f"\\textbf{{{row['EDR']}}}"

        if float(row["F1-score"]) == best_f1_value:
            formatted_df.loc[idx, "F1-score"] = f"\\textbf{{{row['F1-score']}}}"

        if float(row["Precision"]) == best_precision_value:
            formatted_df.loc[idx, "Precision"] = f"\\textbf{{{row['Precision']}}}"

        if float(row["Recall"]) == best_recall_value:
            formatted_df.loc[idx, "Recall"] = f"\\textbf{{{row['Recall']}}}"

    return formatted_df


# Apply bold formatting
df_formatted_domain_shifted = apply_bold_to_best_values_domain_shifted(
    df_domain_shifted
)

# Reorder columns
df_formatted_domain_shifted = df_formatted_domain_shifted[
    ["Method", "Parameter", "EDR", "Precision", "Recall", "F1-score"]
]

# Create styler and generate LaTeX table
styler_domain_shifted = df_formatted_domain_shifted.style.hide(axis="index")


latex_table_domain_shifted = styler_domain_shifted.to_latex(
    caption="Best EDR results for relationship discovery methods on Caltech-256 2-Domain Domain Shifted Synthetic Taxonomy. For each method, the parameter values that yielded the lowest Edge Difference Ratio (EDR) are shown along with the corresponding performance metrics.",
    label="tab:relationship_methods_domain_shifted_results",
    column_format="lccccc",  # Left align method, center align metrics
    position="ht",
    position_float="centering",
    hrules=True,
)

# Save to file
with open(
    "../../thesis/figures/relationship_methods_domain_shifted_results.tex", "w"
) as f:
    f.write(latex_table_domain_shifted)

print(df_domain_shifted.to_string(index=False))

                 Method   EDR F1-score Precision Recall Parameter
                   MCFP 0.744    0.482     0.663  0.379       N/A
     Naive Thresholding 0.700    0.570     0.615  0.531      0.15
   Density Thresholding 0.703    0.499     0.395  0.677      0.50
Relationship Hypothesis 0.696    0.530     0.440  0.665         4
